In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac_cage as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac_cage as training_utils


from scipy import stats

2023-04-26 03:48:19.133891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 03:48:19.310676: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-26 03:48:19.310712: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-26 03:48:20.180442: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-6')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2023-04-26 03:48:21.584929: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-26 03:48:21.584979: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-26 03:48:21.585005: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


In [ ]:
g = tf.random.Generator.from_seed(1)
with strategy.scope():

    tr_data_it,val_data_it,val_data_ho_it, val_data_TSS_it,val_data_TSS_ho_it = \
        training_utils.return_distributed_iterators("gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_5prime",
                                     "gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_5prime_holdout",
                                     "gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_TSS_5prime",
                                     "gs://picard-testing-176520/paired_rampage_atac/262k/genformer_atac_rampage_globalacc_conv_rpgc_TSS_5prime_holdout",
                                     24,
                                     262144,
                                     10,
                                     65536,
                                     2048,
                                     256,
                                     128,
                                     4,
                                     5,
                                     strategy,
                                     options,
                                     0.00,
                                     1024,
                                     True,
                                     True,
                                     True,
                                     False,
                                     g)



In [ ]:
test = next(tr_data_it)

In [ ]:
with strategy.scope():


    model = aformer.aformer(kernel_transformation = 'relu_kernel_transformation',
                             dropout_rate = 0.30,
                             pointwise_dropout_rate=0.1,
                             input_length = 262144,
                             output_length = 2048,
                             final_output_length = 1536,
                             num_heads = 8,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             num_transformer_layers = 6,
                             norm=True,
                             BN_momentum=0.90,
                             max_seq_length = 2048,
                             use_rot_emb = True,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=False,
                             stable_variant=False,
                             inits=None,
                             freeze_conv_layers=False,
                             freeze_BN_layers=False,
                             predict_atac=False,
                             learnable_PE=True,
                             filter_list_seq=[768,896,1024,1152,1280,1536],
                             inits_type='enformer_performer')
    
    def build_step(iterator): #input_batch, model, optimizer, organism, gradient_clip):
        @tf.function(jit_compile=True)
        def test_step(inputs):
            sequence=tf.cast(inputs['sequence'],dtype=tf.bfloat16)
            atac=tf.cast(inputs['atac'],dtype=tf.bfloat16)
            target=tf.cast(inputs['target'],dtype=tf.float32)
            #global_acc=tf.cast(inputs['global_acc'],dtype=tf.bfloat16)         
            input_tuple = sequence,atac#,global_acc

            output = model(input_tuple,
                           training=False)

        for _ in tf.range(1): ## for loop within @tf.fuction for improved TPU performance
            strategy.run(test_step, args=(next(iterator),))
            
            
    build_step(val_data_it)

    print('ran test input')
    #model.load_weights("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_262k_load-True_LR-0.01_T-7_D-0.3_2023-04-13_15:39:43/final/saved_model")
    model.load_weights("gs://picard-testing-176520/genformer_atac_pretrain/models/aformer_hg_mm_rm_rat_262k_load-True_LR-0.01_T-6_D-0.3_2023-04-24_19:58:40/iteration_8/saved_model")
    print('loaded weights')

In [5]:
with strategy.scope():
    metric_dict = {}
    """
    optimizer1 = tf.keras.optimizers.Adam(learning_rate=1.0e-07,
                                          epsilon=1.0e-08)
    optimizer2 = tf.keras.optimizers.Adam(learning_rate=1.0e-07,
                                          epsilon=1.0e-08)
    optimizer3 = tf.keras.optimizers.Adam(learning_rate=1.0e-04,
                                          epsilon=1.0e-08)
    """
    optimizer1 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-08, 
        epsilon=1.0e-14,
        rectify=True)
    optimizer2 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-08, 
        epsilon=1.0e-14,
        rectify=True)
    optimizer3 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-04, 
        epsilon=1.0e-14,
        rectify=True)
    optimizers_in=optimizer1,optimizer2,optimizer3

    loss_fn = tf.keras.losses.Poisson(reduction=tf.keras.losses.Reduction.NONE)
    
    metric_dict["train_loss"] = tf.keras.metrics.Mean("train_loss",
                                                 dtype=tf.float32)
    
    def dist_train_step_cage_only(iterator):    
        @tf.function(jit_compile=True)
        def train_step(inputs):
            sequence=tf.cast(inputs['sequence'],dtype=tf.bfloat16)
            atac=tf.cast(inputs['atac'],dtype=tf.bfloat16)
            target=tf.cast(inputs['target'],dtype=tf.float32)
            mask=tf.cast(inputs['mask'],dtype=tf.float32)
            mask_gathered=tf.cast(inputs['mask_gathered'],dtype=tf.int32)
            peaks=tf.cast(inputs['peaks'],dtype=tf.int32)

            input_tuple = sequence, atac

            with tf.GradientTape() as tape:

                conv_vars = model.stem_conv.trainable_variables + \
                            model.stem_res_conv.trainable_variables + \
                            model.stem_pool.trainable_variables + \
                            model.conv_tower.trainable_variables + \
                            model.stem_conv_atac.trainable_variables + \
                                model.stem_res_conv_atac.trainable_variables + \
                                        model.stem_pool_atac.trainable_variables + \
                                            model.conv_tower_atac.trainable_variables
                
                performer_vars =model.pos_embedding_learned.trainable_variables + \
                                            model.performer.trainable_variables + \
                                                model.final_pointwise_conv.trainable_variables

                heads_vars = model.final_dense_profile_FT.trainable_variables 

                vars_all = conv_vars + performer_vars + heads_vars

                output_profile = model(input_tuple,
                                       training=True)

                output_profile = tf.cast(output_profile,dtype=tf.float32)
                

                cage_loss = tf.reduce_mean(loss_fn(target[:,:,:],
                                                             output_profile[:,:,:])) * (1. / 24)
                loss = cage_loss
            

            gradients = tape.gradient(loss, vars_all)
            """
            gradients, _ = tf.clip_by_global_norm(gradients, 
                                                  0.2)

            optimizer1.apply_gradients(zip(gradients[:len(conv_vars)], 
                                           conv_vars))
            optimizer2.apply_gradients(zip(gradients[len(conv_vars):len(conv_vars+pointwise_vars)], 
                                           pointwise_vars))
            optimizer3.apply_gradients(zip(gradients[len(conv_vars+pointwise_vars):], 
                                           heads_vars))
            metric_dict["train_loss"].update_state(loss)
            """
            return gradients,loss

        for _ in tf.range(1):
            gradients_out,loss = strategy.run(train_step,
                         args=(next(iterator),))
        return gradients_out,loss


In [6]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(2, 10):
        start = time.time()
        if epoch_i == 1:
            build_step(tr_data_it)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                total_params += tf.size(var)
            print('total params: ' + str(total_params))
        

        grads_out,loss = dist_train_step_cage_only(tr_data_it)
        
        print([tf.reduce_sum(tf.cast(tf.math.is_nan(x.values[0]),dtype=tf.float32)) for x in grads_out])
        print(loss)
        #train_step(data_tr)
        
        print('hg_train_loss: ' + str(metric_dict['train_loss'].result().numpy()))
        
        #continue
        '''
        val_step(data_val)
        #val_step(data_val)

        val_losses.append(metric_dict['val_loss'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['val_loss'].result().numpy()))
        #print('hg_val_loss_CAGE: ' + str(metric_dict['val_loss_CAGE'].result().numpy()))
        #print('hg_val_loss_ATAC: ' + str(metric_dict['val_loss_ATAC'].result().numpy()))
        #print('hg_val_cage_pearson: ' + str(metric_dict['CAGE_PearsonR'].result()['PearsonR'].numpy()))
        #print('hg_val_cage_R2: ' + str(metric_dict['CAGE_R2'].result()['R2'].numpy()))
        
        print('hg_val_atac_pearson: ' + str(metric_dict['ATAC_PearsonR'].result()['PearsonR'].numpy()))
        print('hg_val_atac_R2: ' + str(metric_dict['ATAC_R2'].result()['R2'].numpy()))
        
        
        #print('hg_val_atac_baseline_pearson: ' + str(metric_dict['ATAC_PearsonR_baseline'].result()['PearsonR'].numpy()))
        #print('hg_val_atac_baseline_R2: ' + str(metric_dict['ATAC_R2_baseline'].result()['R2'].numpy()))
        """
        val_step_TSS(data_val_TSS)
        #val_step_TSS(data_val_TSS)
        
        y_trues = metric_dict['corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['corr_stats'].result()['gene_map'].numpy()


        figures, corrs_overall = training_utils.make_plots(y_trues,
                                           y_preds, 
                                           cell_types, 
                                           gene_map,50)
        
        cell_spec,gene_spec,cell_spec_sp,gene_spec_sp = corrs_overall
        
        print('cell spec corr: ' + str(cell_spec))
        print('gene_spec_corr:' + str(gene_spec))
        print('cell spec corr_sp: ' + str(cell_spec_sp))
        print('gene_spec_corr_sp:' + str(gene_spec_sp))
        """
        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))
        
        for key, item in metric_dict.items():
            item.reset_state()
        '''

2023-04-26 03:28:53.883583: W tensorflow/core/distributed_runtime/eager/remote_tensor_handle_data.cc:77] Unable to destroy remote tensor handles. If you are running a tf.function, it usually indicates some op in the graph gets an error: 9 root error(s) found.
  (0) RESOURCE_EXHAUSTED: {{function_node __inference_tpu_function_40501}} Attempting to reserve 14.19G at the bottom of memory. That was not possible. There are 14.06G free, 0B reserved, and 14.05G reservable. If fragmentation is eliminated, the maximum reservable bytes would be 14.06G, so compaction wouldn't help.  The nearest obstacle is at 14.05G from the bottom with size 6.25M.
	 [[{{node cluster_tpu_function/_execute_7_0}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[cluster_tpu_function/control_after/_1/_231]]
Hint: If you want to see a list of allocated tensors 

ResourceExhaustedError: Graph execution error:

9 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  Attempting to reserve 14.19G at the bottom of memory. That was not possible. There are 14.06G free, 0B reserved, and 14.05G reservable. If fragmentation is eliminated, the maximum reservable bytes would be 14.06G, so compaction wouldn't help.  The nearest obstacle is at 14.05G from the bottom with size 6.25M.
	 [[{{node cluster_tpu_function/_execute_7_0}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[cluster_tpu_function/control_after/_1/_231]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED:  Attempting to reserve 14.19G at the bottom of memory. That w

In [1]:
sum([tf.reduce_sum(tf.cast(tf.math.is_nan(x.values[0]),dtype=tf.float32)) for x in grads_out])

NameError: name 'grads_out' is not defined

In [13]:
loss

PerReplica:{
  0: <tf.Tensor: shape=(), dtype=float32, numpy=0.056567904>,
  1: <tf.Tensor: shape=(), dtype=float32, numpy=0.057292085>,
  2: <tf.Tensor: shape=(), dtype=float32, numpy=0.05961571>,
  3: <tf.Tensor: shape=(), dtype=float32, numpy=0.057243254>,
  4: <tf.Tensor: shape=(), dtype=float32, numpy=0.055594444>,
  5: <tf.Tensor: shape=(), dtype=float32, numpy=0.06037442>,
  6: <tf.Tensor: shape=(), dtype=float32, numpy=0.93506026>,
  7: <tf.Tensor: shape=(), dtype=float32, numpy=0.057733983>
}